# Plantilla para la Tarea online BDA03

# Nombre del alumno: David Carlon Cembranos

Realiza las tareas que se plantean en cada ejercicio. En algunas tareas deberás completar las celdas que están incompletas en otras añadir nuevas celdas. Se trata de que implementes una serie de consultas con HQL (Hive) y Pig Latin.

Vamos a seguir utilizando el `dataset` de retrasos en vuelos en EEUU de la guía práctica. A modo de recordatorio, en el siguiente apartado, repetimos la explicación del significado de los campos.

# Dataset de retrasos en vuelos

Vamos a usar [este](https://www.kaggle.com/datasets/tylerx/flights-and-airports-data) de Kaggle
para aprender a usar tanto Hive como Pig. Kaggle es un sitio muy popular en ciencia de datos. En este sitio los científicos de datos pueden publicar y compartir sus trabajos. Además también se pueden proponer concursos en los que los participantes compiten en la construcción del mejor modelo para el problema propuesto.

El `dataset` contiene información sobre retrasos en vuelos en EEUU. Hay dos ficheros de interés: `airports.csv` y `flights.csv`.

El primero tiene información sobre los aeropuertos y consta de los siguientes campos:
   * airport_id: identificador del aeropuerto. Numérico, aunque se utilizará un campo `string` en Hive.
   * city: ciudad del aeropuerto.
   * state: estado del aeropuerto.
   * name: nombre del aeropuerto.
   
El fichero `flights` tiene la siguiente estructura:
   * DayofMonth: día del mes del vuelo.
   * DayOfWeek: día de la semana del vuelo.
   * Carrier: Identificador de la compañía aérea.
   * OriginAirportID: Identificador del aeropuerto de origen.
   * DestAirportID: Identificador del aeropuerto de destino.
   * DepDelay: Minutos de retraso en la salida de un vuelo (puede ser negativo si el vuelo sale antes de lo previsto).
   * ArrDelay: Minutos de retraso en la llegada de un vuelo (puede ser negativo si el vuelo sale antes de lo previsto).

El directorio `notebooks` contiene el `archiv.zip` con los dos ficheros. Para descargarlo de Kaggle hay que estar registrado y se ha incluido para que no tengas que registrarte.

## 1.- Realiza el proceso de preparación que se hizo en la guia práctica:

* Crea las celdas y muestra el resultado de su ejecución de la extracción de los ficheros del `dataset` de vuelos.
* Crea la base de datos de Hive y las tablas `airports` y `flights`. Presta atención a cambiar los comentarios y no simplemente copiar los de la guía.
* Carga las tablas y crea consultas de HQL que muestren 10 aeropuertos y 10 vuelos como se hizo en la guía práctica.
* Crea un `script` en Pig Latin que muestre 10 aeropuertos y 10 vuelos como se hizo en la guía práctica.

In [1]:
! apt update

Get:1 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:2 http://archive.ubuntu.com/ubuntu focal InRelease   
Get:3 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:4 http://security.ubuntu.com/ubuntu focal-security/universe amd64 Packages [1176 kB]
Hit:5 http://archive.ubuntu.com/ubuntu focal-backports InRelease
Get:6 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 Packages [1473 kB]
Get:7 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [3372 kB]
Get:8 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 Packages [3850 kB]m
Fetched 10.1 MB in 4s (2878 kB/s)                         m
Reading package lists... Done
Building dependency tree       
Reading state information... Done
207 packages can be upgraded. Run 'apt list --upgradable' to see them.


### Instalamos UNZIP

In [2]:
! apt install unzip

Reading package lists... Done
Building dependency tree       
Reading state information... Done
unzip is already the newest version (6.0-25ubuntu1.1).
0 upgraded, 0 newly installed, 0 to remove and 207 not upgraded.


### Extraemos los archivos CSV

In [3]:
! unzip -j  -o archive.zip airports.csv flights.csv

Archive:  archive.zip
  inflating: airports.csv            
  inflating: flights.csv             


### Mostramos primeras lineas del dataset

In [4]:
! wc -l flights.csv && head flights.csv

2702219 flights.csv












### Copiamos ficheros HDFS

In [5]:
! hdfs dfs -mkdir -p /user/root/flights
! hdfs dfs -put -f ./airports.csv /user/root/flights/
! hdfs dfs -put -f ./flights.csv /user/root/flights/
! hdfs dfs -ls /user/root/flights/

Found 2 items
-rw-r--r--   3 root supergroup      16308 2024-02-15 09:25 /user/root/flights/airports.csv
-rw-r--r--   3 root supergroup   72088113 2024-02-15 09:25 /user/root/flights/flights.csv


## HIVE
### Mostramos  con beeline si hay bases de datos

In [6]:
! beeline -u "jdbc:hive2://localhost:10000" -e "SHOW DATABASES"

Connecting to jdbc:hive2://localhost:10000
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=root_20240215092710_01ae206e-7a6b-4f62-b101-147d7df63212): SHOW DATABASES
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Semantic Analysis Completed (retrial = false)
INFO  : Returning Hive schema: Schema(fieldSchemas:[FieldSchema(name:database_name, type:string, comment:from deserializer)], properties:null)
INFO  : Completed compiling command(queryId=root_20240215092710_01ae206e-7a6b-4f62-b101-147d7df63212); Time taken: 0.712 seconds
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Executing command(queryId=root_20240215092710_01ae206e-7a6b-4f62-b101-147d7df63212): SHOW DATABASES
INFO  : Starting task [Stage-0:DDL] in serial mode
INFO  : Completed executing command(queryId=root_20240215092710_01ae206e-7a6b-4f62-b101-147d7df63212); Ti

### Creamos una nueva tabla

In [7]:
! beeline -u "jdbc:hive2://localhost:10000/" -e "\
CREATE DATABASE IF NOT EXISTS bda03 \
COMMENT 'Base de datos para la tarea BDA03' \
WITH DBPROPERTIES ('Creada por' = 'David Carlon', 'Fecha' = '15/02/2024');"

Connecting to jdbc:hive2://localhost:10000/
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=root_20240215092929_736b8fb5-d27f-4d88-bf70-ad6c20047395): CREATE DATABASE IF NOT EXISTS bda03  COMMENT 'Base de datos para la tarea BDA03'  WITH DBPROPERTIES ('Creada por' = 'David Carlon', 'Fecha' = '15/02/2024')
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Semantic Analysis Completed (retrial = false)
INFO  : Returning Hive schema: Schema(fieldSchemas:null, properties:null)
INFO  : Completed compiling command(queryId=root_20240215092929_736b8fb5-d27f-4d88-bf70-ad6c20047395); Time taken: 0.013 seconds
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Executing command(queryId=root_20240215092929_736b8fb5-d27f-4d88-bf70-ad6c20047395): CREATE DATABASE IF NOT EXISTS bda03  COMMENT 'Base de datos para la tarea BDA03'  WITH DBPROPERTIE

### Creamos tablas airportos y flights

In [8]:
! beeline -u "jdbc:hive2://localhost:10000/bda03" -e "\
DROP TABLE IF EXISTS airports; \
CREATE EXTERNAL TABLE IF NOT EXISTS airports (airportid STRING, city STRING, state STRING, airportname STRING) \
COMMENT 'USA Airports' \
ROW FORMAT DELIMITED FIELDS TERMINATED BY '\,' \
TBLPROPERTIES ('Autor' = 'David Carlon', 'Fecha' = '15/02/2024', 'skip.header.line.count'='1');"

Connecting to jdbc:hive2://localhost:10000/bda03
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=root_20240215093200_44710555-e64c-4f07-aba6-40b512d801b0): DROP TABLE IF EXISTS airports
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Semantic Analysis Completed (retrial = false)
INFO  : Returning Hive schema: Schema(fieldSchemas:null, properties:null)
INFO  : Completed compiling command(queryId=root_20240215093200_44710555-e64c-4f07-aba6-40b512d801b0); Time taken: 0.081 seconds
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Executing command(queryId=root_20240215093200_44710555-e64c-4f07-aba6-40b512d801b0): DROP TABLE IF EXISTS airports
INFO  : Starting task [Stage-0:DDL] in serial mode
INFO  : Completed executing command(queryId=root_20240215093200_44710555-e64c-4f07-aba6-40b512d801b0); Time taken: 0.145 seconds
INFO  : O

In [9]:
! beeline -u "jdbc:hive2://localhost:10000/bda03" -e "\
DROP TABLE IF EXISTS flights; \
CREATE EXTERNAL TABLE IF NOT EXISTS flights (dayofmonth TINYINT, dayofweek TINYINT, carrier STRING, \
    depairportid STRING, arrairportid STRING, depdelay SMALLINT, arrdelay SMALLINT) \
COMMENT 'Flights' \
ROW FORMAT DELIMITED FIELDS TERMINATED BY '\,' \
TBLPROPERTIES ('Autor' = 'David Carlon', 'Fecha' = '15/02/2024', 'skip.header.line.count'='1');"

Connecting to jdbc:hive2://localhost:10000/bda03
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=root_20240215093249_3f5f072e-2aae-4509-b43d-ca41723b3ef7): DROP TABLE IF EXISTS flights
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Semantic Analysis Completed (retrial = false)
INFO  : Returning Hive schema: Schema(fieldSchemas:null, properties:null)
INFO  : Completed compiling command(queryId=root_20240215093249_3f5f072e-2aae-4509-b43d-ca41723b3ef7); Time taken: 0.019 seconds
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Executing command(queryId=root_20240215093249_3f5f072e-2aae-4509-b43d-ca41723b3ef7): DROP TABLE IF EXISTS flights
INFO  : Starting task [Stage-0:DDL] in serial mode
INFO  : Completed executing command(queryId=root_20240215093249_3f5f072e-2aae-4509-b43d-ca41723b3ef7); Time taken: 0.055 seconds
INFO  : OK


### Permisos al directorio 

In [10]:
! hdfs dfs -chmod 777 /user/root/flights

### Cargamos los datos

In [11]:
! beeline -u "jdbc:hive2://localhost:10000/bda03" -e "\
LOAD DATA INPATH '/user/root/flights/airports.csv' OVERWRITE INTO TABLE airports;"

Connecting to jdbc:hive2://localhost:10000/bda03
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=root_20240215093453_1ad8ebb7-4cc4-42c6-bfe1-5459bddfd308): LOAD DATA INPATH '/user/root/flights/airports.csv' OVERWRITE INTO TABLE airports
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Semantic Analysis Completed (retrial = false)
INFO  : Returning Hive schema: Schema(fieldSchemas:null, properties:null)
INFO  : Completed compiling command(queryId=root_20240215093453_1ad8ebb7-4cc4-42c6-bfe1-5459bddfd308); Time taken: 0.03 seconds
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Executing command(queryId=root_20240215093453_1ad8ebb7-4cc4-42c6-bfe1-5459bddfd308): LOAD DATA INPATH '/user/root/flights/airports.csv' OVERWRITE INTO TABLE airports
INFO  : Starting task [Stage-0:MOVE] in serial mode
INFO  : Loading data to table bda03.

In [12]:
! beeline -u "jdbc:hive2://localhost:10000/bda03" -e "\
LOAD DATA INPATH '/user/root/flights/flights.csv' OVERWRITE INTO TABLE flights;"

Connecting to jdbc:hive2://localhost:10000/bda03
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=root_20240215093513_0a4583de-26d4-4fc2-ad2f-a5d6997d5942): LOAD DATA INPATH '/user/root/flights/flights.csv' OVERWRITE INTO TABLE flights
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Semantic Analysis Completed (retrial = false)
INFO  : Returning Hive schema: Schema(fieldSchemas:null, properties:null)
INFO  : Completed compiling command(queryId=root_20240215093513_0a4583de-26d4-4fc2-ad2f-a5d6997d5942); Time taken: 0.019 seconds
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Executing command(queryId=root_20240215093513_0a4583de-26d4-4fc2-ad2f-a5d6997d5942): LOAD DATA INPATH '/user/root/flights/flights.csv' OVERWRITE INTO TABLE flights
INFO  : Starting task [Stage-0:MOVE] in serial mode
INFO  : Loading data to table bda03.fli

### Mostramos 10 aeropuertos

In [13]:
! ! beeline -u "jdbc:hive2://localhost:10000/bda03" -e "\
SELECT * FROM airports LIMIT 10"

Connecting to jdbc:hive2://localhost:10000/bda03
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=root_20240215093634_f31b8be4-1475-46ab-ada7-01f72d27ce5c): SELECT * FROM airports LIMIT 10
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Semantic Analysis Completed (retrial = false)
INFO  : Returning Hive schema: Schema(fieldSchemas:[FieldSchema(name:airports.airportid, type:string, comment:null), FieldSchema(name:airports.city, type:string, comment:null), FieldSchema(name:airports.state, type:string, comment:null), FieldSchema(name:airports.airportname, type:string, comment:null)], properties:null)
INFO  : Completed compiling command(queryId=root_20240215093634_f31b8be4-1475-46ab-ada7-01f72d27ce5c); Time taken: 0.706 seconds
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Executing command(queryId=root_20240215093634_f31b8be

### Mostramos 10 vuelos

In [14]:
! beeline -u "jdbc:hive2://localhost:10000/bda03" -e "\
SELECT * FROM flights LIMIT 10"

Connecting to jdbc:hive2://localhost:10000/bda03
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=root_20240215093710_8317c8b0-e520-4a8a-ae4b-caeadb0b8dd1): SELECT * FROM flights LIMIT 10
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Semantic Analysis Completed (retrial = false)
INFO  : Returning Hive schema: Schema(fieldSchemas:[FieldSchema(name:flights.dayofmonth, type:tinyint, comment:null), FieldSchema(name:flights.dayofweek, type:tinyint, comment:null), FieldSchema(name:flights.carrier, type:string, comment:null), FieldSchema(name:flights.depairportid, type:string, comment:null), FieldSchema(name:flights.arrairportid, type:string, comment:null), FieldSchema(name:flights.depdelay, type:smallint, comment:null), FieldSchema(name:flights.arrdelay, type:smallint, comment:null)], properties:null)
INFO  : Completed compiling command(queryId=root_2024

## PIG
### Mostramos 10 aeropuertos y 10 vuelos 

In [15]:
%%writefile flights.pig

-- registramos la librería PiggyBank para poder usar la función de carga CSVExcelStorage.
REGISTER piggybank.jar

/*
Leemos el fichero de airports.csv.
Usamos el loader CSVExcelStorage indicando el delimitador (,) y que se debe excluir la cabecera.
*/

AIRPORTS = LOAD '$airports_file' USING
       org.apache.pig.piggybank.storage.CSVExcelStorage(',', 'NO_MULTILINE', 'UNIX', 'SKIP_INPUT_HEADER')
       AS (airportid:chararray, city:chararray, state:chararray, airportname:chararray);

-- Leemos el fichero fligths.csv
FLIGHTS = LOAD '$flights_file' USING
       org.apache.pig.piggybank.storage.CSVExcelStorage(',', 'NO_MULTILINE', 'UNIX', 'SKIP_INPUT_HEADER')
       AS (dayofmonth:int, dayofweek:int, carrier:chararray, 
               depairportid:chararray, arrairportid:chararray, depdelay:int, arrdelay:int);

-- Probamos que podemos recuperar datos.
      
-- Nos quedamos con 10 aeropuertos
AIRPORTS_10 = LIMIT AIRPORTS 10;

-- Mostramos 10 aeropuertos
DUMP AIRPORTS_10;

-- Hacemos lo mismo con los vuelos
FLIGHTS_10 = LIMIT FLIGHTS 10;
DUMP FLIGHTS_10;

Writing flights.pig


In [16]:
! pig -x local -f flights.pig -param airports_file='airports.csv' -param flights_file='flights.csv' -param output_dir='pig/output/flights'


2024-02-15 09:40:43,738 INFO pig.ExecTypeProvider: Trying ExecType : LOCAL
2024-02-15 09:40:43,739 INFO pig.ExecTypeProvider: Picked LOCAL as the ExecType
2024-02-15 09:40:43,767 [main] INFO  org.apache.pig.Main - Apache Pig version 0.17.0 (r1797386) compiled Jun 02 2017, 15:41:58
2024-02-15 09:40:43,767 [main] INFO  org.apache.pig.Main - Logging error messages to: /media/notebooks/TAREA03/notebooks/pig_1707986443766.log
2024-02-15 09:40:43,774 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - user.name is deprecated. Instead, use mapreduce.job.user.name
2024-02-15 09:40:43,844 [main] INFO  org.apache.pig.impl.util.Utils - Default bootup file /root/.pigbootup not found
2024-02-15 09:40:43,878 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.job.tracker is deprecated. Instead, use mapreduce.jobtracker.address
2024-02-15 09:40:43,878 [main] INFO  org.apache.pig.backend.hadoop.executionengine.HExecutionEngine - Connecting to hadoop file system at: file:

## 2.- Con una consulta de HQL muestra: La cinco compañías que más vuelos retrasados tienen.

* El campo `carrier` contiene la compañía aérea.
* Vamos a considerar que un vuelo llega con retraso cuando el vuelo llega más de 15 minutos tarde (campo `arrdelay` > 15).

Se espera el siguiente resultado:

![solución 2](./img/2.png)

### Utilizamos beeline para conectar con la base de datos de Hive "bda03". Ejecutamos la consulta que selecciona la compañia aerea y cuenta los vuelos retrasados de llegada con mas de 15 minutos. Ordemos los resultados segun el nuemro de vuelos retrasados y mostramos las 5 compañias con mas vuelos retrasados.

In [44]:
! beeline -u "jdbc:hive2://localhost:10000/bda03" -e "\
SELECT carrier, COUNT(*) as delayed_flights FROM flights WHERE arrdelay > 15 GROUP BY carrier \
ORDER BY delayed_flights DESC LIMIT 5;"

Connecting to jdbc:hive2://localhost:10000/bda03
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=root_20240215095526_416ca2e5-74aa-4176-a789-25d7af248d5b): SELECT carrier, COUNT(*) as delayed_flights FROM flights WHERE arrdelay > 15 GROUP BY carrier  ORDER BY delayed_flights DESC LIMIT 5
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Semantic Analysis Completed (retrial = false)
INFO  : Returning Hive schema: Schema(fieldSchemas:[FieldSchema(name:carrier, type:string, comment:null), FieldSchema(name:delayed_flights, type:bigint, comment:null)], properties:null)
INFO  : Completed compiling command(queryId=root_20240215095526_416ca2e5-74aa-4176-a789-25d7af248d5b); Time taken: 0.102 seconds
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Executing command(queryId=root_20240215095526_416ca2e5-74aa-4176-a789-25d7af248d5b): SELE

## 3.- Con una consulta de HQL muestra: Las 5 compañías que mejor recuperación de tiempo en vuelo tienen.

* Se considera que se ha recuperado el tiempo de un vuelo cuando habiendo salido con retraso (`depdelay` > 15), llega sin retraso (`arraydelay` <= 15).
* Se trata de que muestres las 5 compañías que han recuperado el tiempo en un mayor porcentaje de vuelos que salieron retrasados.

El resultado esperado es el siguiente:

![solución 3](./img/3.png)

### Utilizamos Beeline para conectar con la base de datos Hive 'bda03' . Ejecutamos la consulta que selecciona la compañía aérea y calcula el porcentaje de vuelos recuperados, es decir, los vuelos que tuvieron un retraso en la salida (depdelay) de más de 15 minutos, pero llegaron con un retraso de 15 minutos o menos. Los resultados se ordenan en orden descendente según el porcentaje de vuelos recuperados y se limitan a las 5 primeras compañías aéreas con el mayor porcentaje de vuelos recuperados.

In [45]:
! beeline -u "jdbc:hive2://localhost:10000/bda03" -e "\
SELECT carrier, COUNT(CASE WHEN arrdelay <= 15 THEN 1 END) / COUNT(*) as percent_recovered \
FROM flights WHERE depdelay > 15 GROUP BY carrier ORDER BY percent_recovered DESC LIMIT 5"

Connecting to jdbc:hive2://localhost:10000/bda03
Connected to: Apache Hive (version 3.1.2)
Driver: Hive JDBC (version 3.1.2)
Transaction isolation: TRANSACTION_REPEATABLE_READ
INFO  : Compiling command(queryId=root_20240215095641_c4d6347a-db82-4eea-92a9-4141962ce020): SELECT carrier, COUNT(CASE WHEN arrdelay <= 15 THEN 1 END) / COUNT(*) as percent_recovered  FROM flights WHERE depdelay > 15 GROUP BY carrier ORDER BY percent_recovered DESC LIMIT 5
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Semantic Analysis Completed (retrial = false)
INFO  : Returning Hive schema: Schema(fieldSchemas:[FieldSchema(name:carrier, type:string, comment:null), FieldSchema(name:percent_recovered, type:double, comment:null)], properties:null)
INFO  : Completed compiling command(queryId=root_20240215095641_c4d6347a-db82-4eea-92a9-4141962ce020); Time taken: 0.126 seconds
INFO  : Concurrency mode is disabled, not creating a lock manager
INFO  : Executing command(queryId=root_2024021

## 4.- Resuelve el ejercicio 2 con Pig Latin

El resultado esperado es:

![solución 4](./img/4.png)

In [119]:
%%writefile delay_flights.pig

-- registramos la librería PiggyBank para poder usar la función de carga CSVExcelStorage.
REGISTER piggybank.jar

-- Leemos el fichero fligths.csv

FLIGHTS = LOAD '$flights_file' USING
       org.apache.pig.piggybank.storage.CSVExcelStorage(',', 'NO_MULTILINE', 'UNIX', 'SKIP_INPUT_HEADER')
       AS (dayofmonth:int, dayofweek:int, carrier:chararray, 
               depairportid:chararray, arrairportid:chararray, depdelay:int, arrdelay:int);

-- Filtramos las llegadas con retraso, seleccionando aquellos vuelos con un retraso en la llegada mayor a 15 minutos.
OPERATIONS = filter FLIGHTS by arrdelay > 15;

-- Agrupamos los vuelos filtrados por la compañía aérea.

TOTAL_OPERATIONS = GROUP OPERATIONS BY carrier;

-- Mostramos el esquema resultante.
DESCRIBE TOTAL_OPERATIONS;

-- Renombramos los campos y se cuenta el número de vuelos retrasados para cada compañía aérea.
TOTAL_OPERATIONS = FOREACH TOTAL_OPERATIONS GENERATE group AS carrier, COUNT(OPERATIONS) AS delayed_flights;

-- Ordenamos de forma descendente por vuelos
TOTAL_OPERATIONS = ORDER TOTAL_OPERATIONS BY delayed_flights DESC;

-- Limitamos a 5 aeropuertos
TOP5_TOTAL_OPERATIONS = LIMIT TOTAL_OPERATIONS 5;

-- Mostramos los resultados.
DUMP TOP5_TOTAL_OPERATIONS;

Writing delay_flights.pig


### Ejecutamos el script "delay_flights.pig".

In [120]:
! pig -x local -f delay_flights.pig -param airports_file='airports.csv' -param flights_file='flights.csv' -param output_dir='pig/output/delayed_flights'


2024-02-15 10:28:32,142 INFO pig.ExecTypeProvider: Trying ExecType : LOCAL
2024-02-15 10:28:32,143 INFO pig.ExecTypeProvider: Picked LOCAL as the ExecType
2024-02-15 10:28:32,169 [main] INFO  org.apache.pig.Main - Apache Pig version 0.17.0 (r1797386) compiled Jun 02 2017, 15:41:58
2024-02-15 10:28:32,169 [main] INFO  org.apache.pig.Main - Logging error messages to: /media/notebooks/TAREA03/notebooks/pig_1707989312168.log
2024-02-15 10:28:32,176 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - user.name is deprecated. Instead, use mapreduce.job.user.name
2024-02-15 10:28:32,247 [main] INFO  org.apache.pig.impl.util.Utils - Default bootup file /root/.pigbootup not found
2024-02-15 10:28:32,281 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.job.tracker is deprecated. Instead, use mapreduce.jobtracker.address
2024-02-15 10:28:32,282 [main] INFO  org.apache.pig.backend.hadoop.executionengine.HExecutionEngine - Connecting to hadoop file system at: file:

2024-02-15 10:28:33,229 [LocalJobRunner Map Task Executor #0] INFO  org.apache.hadoop.mapred.Task -  Using ResourceCalculatorProcessTree : [ ]
2024-02-15 10:28:33,232 [LocalJobRunner Map Task Executor #0] INFO  org.apache.hadoop.mapred.MapTask - Processing split: Number of splits :1
Total Length = 33554432
Input split[0]:
   Length = 33554432
   ClassName: org.apache.hadoop.mapreduce.lib.input.FileSplit
   Locations:

-----------------------

2024-02-15 10:28:33,239 [LocalJobRunner Map Task Executor #0] INFO  org.apache.pig.backend.hadoop.executionengine.mapReduceLayer.PigRecordReader - Current split being processed file:/media/notebooks/TAREA03/notebooks/flights.csv:0+33554432
2024-02-15 10:28:33,258 [LocalJobRunner Map Task Executor #0] INFO  org.apache.hadoop.mapred.MapTask - (EQUATOR) 0 kvi 26214396(104857584)
2024-02-15 10:28:33,258 [LocalJobRunner Map Task Executor #0] INFO  org.apache.hadoop.mapred.MapTask - mapreduce.task.io.sort.mb: 100
2024-02-15 10:28:33,259 [LocalJobRunner 

2024-02-15 10:28:36,990 [main] INFO  org.apache.pig.backend.hadoop.executionengine.mapReduceLayer.MapReduceLauncher - 12% complete
2024-02-15 10:28:36,990 [main] INFO  org.apache.pig.backend.hadoop.executionengine.mapReduceLayer.MapReduceLauncher - Running jobs are [job_local639595835_0001]
2024-02-15 10:28:39,610 [LocalJobRunner Map Task Executor #0] INFO  org.apache.hadoop.mapred.LocalJobRunner - 
2024-02-15 10:28:39,610 [LocalJobRunner Map Task Executor #0] INFO  org.apache.hadoop.mapred.MapTask - Starting flush of map output
2024-02-15 10:28:39,610 [LocalJobRunner Map Task Executor #0] INFO  org.apache.hadoop.mapred.MapTask - Spilling map output
2024-02-15 10:28:39,610 [LocalJobRunner Map Task Executor #0] INFO  org.apache.hadoop.mapred.MapTask - bufstart = 0; bufend = 2384890; bufvoid = 104857600
2024-02-15 10:28:39,610 [LocalJobRunner Map Task Executor #0] INFO  org.apache.hadoop.mapred.MapTask - kvstart = 26214396(104857584); kvend = 25260444(101041776); length = 953953/6553600


2024-02-15 10:28:40,256 [pool-4-thread-1] INFO  org.apache.hadoop.mapred.Task - Task:attempt_local639595835_0001_r_000000_0 is done. And is in the process of committing
2024-02-15 10:28:40,258 [pool-4-thread-1] INFO  org.apache.hadoop.mapred.LocalJobRunner - 3 / 3 copied.
2024-02-15 10:28:40,258 [pool-4-thread-1] INFO  org.apache.hadoop.mapred.Task - Task attempt_local639595835_0001_r_000000_0 is allowed to commit now
2024-02-15 10:28:40,260 [pool-4-thread-1] INFO  org.apache.hadoop.mapreduce.lib.output.FileOutputCommitter - Saved output of task 'attempt_local639595835_0001_r_000000_0' to file:/tmp/temp232622680/tmp-696106867
2024-02-15 10:28:40,260 [pool-4-thread-1] INFO  org.apache.hadoop.mapred.LocalJobRunner - reduce > reduce
2024-02-15 10:28:40,260 [pool-4-thread-1] INFO  org.apache.hadoop.mapred.Task - Task 'attempt_local639595835_0001_r_000000_0' done.
2024-02-15 10:28:40,261 [pool-4-thread-1] INFO  org.apache.hadoop.mapred.Task - Final Counters for attempt_local639595835_0001_r

2024-02-15 10:28:40,693 [main] INFO  org.apache.pig.backend.hadoop.executionengine.mapReduceLayer.MapReduceLauncher - HadoopJobId: job_local635709360_0002
2024-02-15 10:28:40,693 [main] INFO  org.apache.pig.backend.hadoop.executionengine.mapReduceLayer.MapReduceLauncher - Processing aliases TOTAL_OPERATIONS
2024-02-15 10:28:40,693 [main] INFO  org.apache.pig.backend.hadoop.executionengine.mapReduceLayer.MapReduceLauncher - detailed locations: M: TOTAL_OPERATIONS[26,19] C:  R: 
2024-02-15 10:28:40,693 [main] INFO  org.apache.pig.backend.hadoop.executionengine.mapReduceLayer.MapReduceLauncher - 50% complete
2024-02-15 10:28:40,694 [main] WARN  org.apache.hadoop.metrics2.impl.MetricsSystemImpl - JobTracker metrics system already initialized!
2024-02-15 10:28:40,695 [main] WARN  org.apache.hadoop.metrics2.impl.MetricsSystemImpl - JobTracker metrics system already initialized!
2024-02-15 10:28:40,696 [main] WARN  org.apache.hadoop.metrics2.impl.MetricsSystemImpl - JobTracker metrics system 

2024-02-15 10:28:40,957 [main] INFO  org.apache.pig.backend.hadoop.executionengine.mapReduceLayer.MapReduceLauncher - HadoopJobId: job_local1047169466_0003
2024-02-15 10:28:40,957 [main] INFO  org.apache.pig.backend.hadoop.executionengine.mapReduceLayer.MapReduceLauncher - Processing aliases TOTAL_OPERATIONS
2024-02-15 10:28:40,957 [main] INFO  org.apache.pig.backend.hadoop.executionengine.mapReduceLayer.MapReduceLauncher - detailed locations: M: TOTAL_OPERATIONS[26,19] C:  R: 
2024-02-15 10:28:40,958 [main] INFO  org.apache.pig.backend.hadoop.executionengine.mapReduceLayer.MapReduceLauncher - 75% complete
2024-02-15 10:28:40,959 [main] WARN  org.apache.hadoop.metrics2.impl.MetricsSystemImpl - JobTracker metrics system already initialized!
2024-02-15 10:28:40,960 [main] WARN  org.apache.hadoop.metrics2.impl.MetricsSystemImpl - JobTracker metrics system already initialized!
2024-02-15 10:28:40,960 [main] WARN  org.apache.hadoop.metrics2.impl.MetricsSystemImpl - JobTracker metrics system

2024-02-15 10:28:41,213 [main] INFO  org.apache.pig.backend.hadoop.executionengine.mapReduceLayer.MapReduceLauncher - HadoopJobId: job_local1434056528_0004
2024-02-15 10:28:41,213 [main] INFO  org.apache.pig.backend.hadoop.executionengine.mapReduceLayer.MapReduceLauncher - Processing aliases TOTAL_OPERATIONS
2024-02-15 10:28:41,213 [main] INFO  org.apache.pig.backend.hadoop.executionengine.mapReduceLayer.MapReduceLauncher - detailed locations: M: TOTAL_OPERATIONS[26,19] C:  R: 
2024-02-15 10:28:41,214 [main] WARN  org.apache.hadoop.metrics2.impl.MetricsSystemImpl - JobTracker metrics system already initialized!
2024-02-15 10:28:41,215 [main] WARN  org.apache.hadoop.metrics2.impl.MetricsSystemImpl - JobTracker metrics system already initialized!
2024-02-15 10:28:41,215 [main] WARN  org.apache.hadoop.metrics2.impl.MetricsSystemImpl - JobTracker metrics system already initialized!
2024-02-15 10:28:41,218 [main] INFO  org.apache.pig.backend.hadoop.executionengine.mapReduceLayer.MapReduceLa

## 5.- Resuelve el ejercicio 3 con Pig Latin

Se espera el siguiente resultado:

![solución 5](./img/5.png)

In [121]:
%%writefile recovery.pig

-- Leemos el archivo flights.csv.
flights_data = LOAD '$flights_file' USING org.apache.pig.piggybank.storage.CSVExcelStorage(',', 'NO_MULTILINE', 'UNIX', 'SKIP_INPUT_HEADER')
       AS (dayofmonth:int, dayofweek:int, carrier:chararray, 
              depairportid:chararray, arrairportid:chararray, depdelay:int, arrdelay:int);

-- Filtramos de vuelos con salida retrasada y vuelos recuperados.
delayed_flights = FILTER flights_data BY depdelay > 15;
recovered_flights = FILTER flights_data BY depdelay > 15 AND arrdelay <= 15;

-- Agrupamos los vuelos con salida retrasada por compañía aérea.
grouped_delayed_flights = GROUP delayed_flights BY carrier;
delayed_count = FOREACH grouped_delayed_flights GENERATE
    group AS carrier,
    COUNT(delayed_flights) AS delayed_count;

-- Agrupamos los vuelos recuperados por compañía aérea.
grouped_recovered_flights = GROUP recovered_flights BY carrier;
recovered_count = FOREACH grouped_recovered_flights GENERATE
    group AS carrier,
    COUNT(recovered_flights) AS recovered_count;

-- Mostramos el esquema de las relaciones resultantes.
DESCRIBE grouped_delayed_flights;
DESCRIBE grouped_recovered_flights;

-- Unimos los conteos de vuelos con salida retrasada y vuelos recuperados.
joined_counts = JOIN delayed_count BY carrier, recovered_count BY carrier;

-- Cálculamos el porcentaje de vuelos recuperados.
percent_recovered = FOREACH joined_counts GENERATE
    delayed_count::carrier AS carrier,
    recovered_count / (float)delayed_count::delayed_count AS percent_recovered;

-- Ordenamos descendente por el porcentaje de vuelos recuperados.
ordered_flights = ORDER percent_recovered BY percent_recovered DESC;

-- Limitamos de resultados a los 5 primeros.
limited_flights = LIMIT ordered_flights 5;

-- Mostramos de los resultados finales.
DUMP limited_flights;


Writing recovery.pig


### Ejecutamos el script recovery.pig

In [122]:
! pig -x local -f recovery.pig  -param flights_file='flights.csv' -param output_dir='pig/output/percent_recovered'


2024-02-15 10:30:32,788 INFO pig.ExecTypeProvider: Trying ExecType : LOCAL
2024-02-15 10:30:32,788 INFO pig.ExecTypeProvider: Picked LOCAL as the ExecType
2024-02-15 10:30:32,814 [main] INFO  org.apache.pig.Main - Apache Pig version 0.17.0 (r1797386) compiled Jun 02 2017, 15:41:58
2024-02-15 10:30:32,814 [main] INFO  org.apache.pig.Main - Logging error messages to: /media/notebooks/TAREA03/notebooks/pig_1707989432813.log
2024-02-15 10:30:32,822 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - user.name is deprecated. Instead, use mapreduce.job.user.name
2024-02-15 10:30:32,893 [main] INFO  org.apache.pig.impl.util.Utils - Default bootup file /root/.pigbootup not found
2024-02-15 10:30:32,928 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.job.tracker is deprecated. Instead, use mapreduce.jobtracker.address
2024-02-15 10:30:32,928 [main] INFO  org.apache.pig.backend.hadoop.executionengine.HExecutionEngine - Connecting to hadoop file system at: file:

2024-02-15 10:30:33,802 [JobControl] INFO  org.apache.hadoop.mapreduce.JobSubmitter - Submitting tokens for job: job_local1659756285_0001
2024-02-15 10:30:33,802 [JobControl] INFO  org.apache.hadoop.mapreduce.JobSubmitter - Executing with tokens: []
2024-02-15 10:30:33,862 [JobControl] INFO  org.apache.hadoop.mapreduce.Job - The url to track the job: http://localhost:8080/
2024-02-15 10:30:33,862 [Thread-6] INFO  org.apache.hadoop.mapred.LocalJobRunner - OutputCommitter set in config null
2024-02-15 10:30:33,876 [Thread-6] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.reduce.tasks is deprecated. Instead, use mapreduce.job.reduces
2024-02-15 10:30:33,876 [Thread-6] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.job.tracker is deprecated. Instead, use mapreduce.jobtracker.address
2024-02-15 10:30:33,876 [Thread-6] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.job.reduce.markreset.buffer.percent is deprecated. Instead, use mapreduce.red

2024-02-15 10:30:41,134 [LocalJobRunner Map Task Executor #0] INFO  org.apache.hadoop.mapred.LocalJobRunner - 
2024-02-15 10:30:41,134 [LocalJobRunner Map Task Executor #0] INFO  org.apache.hadoop.mapred.MapTask - Starting flush of map output
2024-02-15 10:30:41,134 [LocalJobRunner Map Task Executor #0] INFO  org.apache.hadoop.mapred.MapTask - Spilling map output
2024-02-15 10:30:41,134 [LocalJobRunner Map Task Executor #0] INFO  org.apache.hadoop.mapred.MapTask - bufstart = 0; bufend = 2854600; bufvoid = 104857600
2024-02-15 10:30:41,134 [LocalJobRunner Map Task Executor #0] INFO  org.apache.hadoop.mapred.MapTask - kvstart = 26214396(104857584); kvend = 25072560(100290240); length = 1141837/6553600
2024-02-15 10:30:41,239 [LocalJobRunner Map Task Executor #0] INFO  org.apache.hadoop.mapred.MapTask - Finished spill 0
2024-02-15 10:30:41,240 [LocalJobRunner Map Task Executor #0] INFO  org.apache.hadoop.mapred.Task - Task:attempt_local1659756285_0001_m_000001_0 is done. And is in the pro

2024-02-15 10:30:41,850 [pool-4-thread-1] INFO  org.apache.hadoop.mapred.Task -  Using ResourceCalculatorProcessTree : [ ]
2024-02-15 10:30:41,851 [pool-4-thread-1] INFO  org.apache.hadoop.mapred.ReduceTask - Using ShuffleConsumerPlugin: org.apache.hadoop.mapreduce.task.reduce.Shuffle@726de6d7
2024-02-15 10:30:41,852 [pool-4-thread-1] WARN  org.apache.hadoop.metrics2.impl.MetricsSystemImpl - JobTracker metrics system already initialized!
2024-02-15 10:30:41,861 [pool-4-thread-1] INFO  org.apache.hadoop.mapreduce.task.reduce.MergeManagerImpl - MergerManager: memoryLimit=652528832, maxSingleShuffleLimit=163132208, mergeThreshold=430669056, ioSortFactor=10, memToMemMergeOutputsThreshold=10
2024-02-15 10:30:41,862 [EventFetcher for fetching Map Completion Events] INFO  org.apache.hadoop.mapreduce.task.reduce.EventFetcher - attempt_local1659756285_0001_r_000000_0 Thread started: EventFetcher for fetching Map Completion Events
2024-02-15 10:30:41,875 [localfetcher#1] INFO  org.apache.hadoop.

2024-02-15 10:30:42,097 [main] INFO  org.apache.pig.backend.hadoop.executionengine.mapReduceLayer.MapReduceLauncher - 20% complete
2024-02-15 10:30:42,099 [main] WARN  org.apache.hadoop.metrics2.impl.MetricsSystemImpl - JobTracker metrics system already initialized!
2024-02-15 10:30:42,105 [main] WARN  org.apache.hadoop.metrics2.impl.MetricsSystemImpl - JobTracker metrics system already initialized!
2024-02-15 10:30:42,106 [main] INFO  org.apache.hadoop.conf.Configuration.deprecation - mapred.map.tasks is deprecated. Instead, use mapreduce.job.maps
2024-02-15 10:30:42,106 [main] WARN  org.apache.hadoop.metrics2.impl.MetricsSystemImpl - JobTracker metrics system already initialized!
2024-02-15 10:30:42,121 [main] INFO  org.apache.pig.tools.pigstats.mapreduce.MRScriptState - Pig script settings are added to the job
2024-02-15 10:30:42,121 [main] INFO  org.apache.pig.backend.hadoop.executionengine.mapReduceLayer.JobControlCompiler - mapred.job.reduce.markreset.buffer.percent is not set, s

2024-02-15 10:30:42,254 [pool-9-thread-1] INFO  org.apache.hadoop.mapred.LocalJobRunner - 2 / 2 copied.
2024-02-15 10:30:42,254 [pool-9-thread-1] INFO  org.apache.hadoop.mapred.Task - Task attempt_local1254809559_0002_r_000000_0 is allowed to commit now
2024-02-15 10:30:42,255 [pool-9-thread-1] INFO  org.apache.hadoop.mapreduce.lib.output.FileOutputCommitter - Saved output of task 'attempt_local1254809559_0002_r_000000_0' to file:/tmp/temp1453782073/tmp-1569116790
2024-02-15 10:30:42,256 [pool-9-thread-1] INFO  org.apache.hadoop.mapred.LocalJobRunner - reduce > reduce
2024-02-15 10:30:42,256 [pool-9-thread-1] INFO  org.apache.hadoop.mapred.Task - Task 'attempt_local1254809559_0002_r_000000_0' done.
2024-02-15 10:30:42,256 [pool-9-thread-1] INFO  org.apache.hadoop.mapred.Task - Final Counters for attempt_local1254809559_0002_r_000000_0: Counters: 24
	File System Counters
		FILE: Number of bytes read=72104914
		FILE: Number of bytes written=1260219
		FILE: Number of read operations=0
		F

2024-02-15 10:30:42,650 [main] INFO  org.apache.pig.backend.hadoop.executionengine.mapReduceLayer.MapReduceLauncher - HadoopJobId: job_local1079352555_0003
2024-02-15 10:30:42,651 [main] INFO  org.apache.pig.backend.hadoop.executionengine.mapReduceLayer.MapReduceLauncher - Processing aliases ordered_flights
2024-02-15 10:30:42,651 [main] INFO  org.apache.pig.backend.hadoop.executionengine.mapReduceLayer.MapReduceLauncher - detailed locations: M: ordered_flights[36,18] C:  R: 
2024-02-15 10:30:42,651 [main] INFO  org.apache.pig.backend.hadoop.executionengine.mapReduceLayer.MapReduceLauncher - 60% complete
2024-02-15 10:30:42,652 [main] WARN  org.apache.hadoop.metrics2.impl.MetricsSystemImpl - JobTracker metrics system already initialized!
2024-02-15 10:30:42,653 [main] WARN  org.apache.hadoop.metrics2.impl.MetricsSystemImpl - JobTracker metrics system already initialized!
2024-02-15 10:30:42,653 [main] WARN  org.apache.hadoop.metrics2.impl.MetricsSystemImpl - JobTracker metrics system a

2024-02-15 10:30:42,911 [main] INFO  org.apache.pig.backend.hadoop.executionengine.mapReduceLayer.MapReduceLauncher - HadoopJobId: job_local764858715_0004
2024-02-15 10:30:42,911 [main] INFO  org.apache.pig.backend.hadoop.executionengine.mapReduceLayer.MapReduceLauncher - Processing aliases ordered_flights
2024-02-15 10:30:42,911 [main] INFO  org.apache.pig.backend.hadoop.executionengine.mapReduceLayer.MapReduceLauncher - detailed locations: M: ordered_flights[36,18] C:  R: 
2024-02-15 10:30:42,912 [main] INFO  org.apache.pig.backend.hadoop.executionengine.mapReduceLayer.MapReduceLauncher - 80% complete
2024-02-15 10:30:42,912 [main] WARN  org.apache.hadoop.metrics2.impl.MetricsSystemImpl - JobTracker metrics system already initialized!
2024-02-15 10:30:42,913 [main] WARN  org.apache.hadoop.metrics2.impl.MetricsSystemImpl - JobTracker metrics system already initialized!
2024-02-15 10:30:42,914 [main] WARN  org.apache.hadoop.metrics2.impl.MetricsSystemImpl - JobTracker metrics system al

2024-02-15 10:30:43,170 [main] INFO  org.apache.pig.backend.hadoop.executionengine.mapReduceLayer.MapReduceLauncher - HadoopJobId: job_local408125882_0005
2024-02-15 10:30:43,170 [main] INFO  org.apache.pig.backend.hadoop.executionengine.mapReduceLayer.MapReduceLauncher - Processing aliases ordered_flights
2024-02-15 10:30:43,170 [main] INFO  org.apache.pig.backend.hadoop.executionengine.mapReduceLayer.MapReduceLauncher - detailed locations: M: ordered_flights[36,18] C:  R: 
2024-02-15 10:30:43,171 [main] WARN  org.apache.hadoop.metrics2.impl.MetricsSystemImpl - JobTracker metrics system already initialized!
2024-02-15 10:30:43,172 [main] WARN  org.apache.hadoop.metrics2.impl.MetricsSystemImpl - JobTracker metrics system already initialized!
2024-02-15 10:30:43,173 [main] WARN  org.apache.hadoop.metrics2.impl.MetricsSystemImpl - JobTracker metrics system already initialized!
2024-02-15 10:30:43,175 [main] INFO  org.apache.pig.backend.hadoop.executionengine.mapReduceLayer.MapReduceLaunc